In [47]:
import pandas as pd
import numpy as np
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math 
import seaborn as sns
import matplotlib.colors as mcolors
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.formula.api import mixedlm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import matplotlib.pyplot as mpl
import matplotlib
from scipy.stats import spearmanr

colors = list(mcolors.TABLEAU_COLORS.keys())*2

parentDirectory = os.path.abspath(os.path.join(os.path.join(os.getcwd(), os.pardir), os.pardir))
DATA_DIR = parentDirectory +'/data/'
FIGURES_DIR = parentDirectory +'/figures/'

full_names = {
    'AU': 'Australia',
    'BR': 'Brazil',
    'CA': 'Canada',
    'FR': 'France',
    'DE': 'Germany',
    'IN': 'India',
    'IT': 'Italy',
    'MX': 'Mexico',
    'ES': 'Spain',
    'GB': 'United Kingdom',
    'US': 'United States',
    'DK': 'Denmark'
}

event_dicts = [{'country': 'AU',
  'start_md_1': '2020-03-27',
  'end_md_1': '2020-06-07',
  'start_md_2': np.nan},
 {'country': 'BR',
  'start_md_1': '2020-03-23',
  'end_md_1': '2020-08-09',
  'start_md_2': np.nan},
 {'country': 'CA',
  'start_md_1': '2020-03-19',
  'end_md_1': '2020-06-21',
  'start_md_2': '2020-10-12'},
 {'country': 'DE',
  'start_md_1': '2020-03-21',
  'end_md_1': '2020-05-09',
  'start_md_2': '2020-12-18'},
 {'country': 'DK',
  'start_md_1': '2020-03-17',
  'end_md_1': '2020-05-07',
  'start_md_2': np.nan},
 {'country': 'ES',
  'start_md_1': '2020-03-17',
  'end_md_1': '2020-06-14',
  'start_md_2': '2020-11-07'},
 {'country': 'FR',
  'start_md_1': '2020-03-18',
  'end_md_1': '2020-06-08',
  'start_md_2': '2020-11-01'},
 {'country': 'GB',
  'start_md_1': '2020-03-23',
  'end_md_1': '2020-08-03',
  'start_md_2': '2020-10-21'},
 {'country': 'IN',
  'start_md_1': '2020-03-24',
  'end_md_1': '2020-10-29',
  'start_md_2': np.nan},
 {'country': 'IT',
  'start_md_1': '2020-03-11',
  'end_md_1': '2020-06-06',
  'start_md_2': '2020-11-06'},
 {'country': 'JP',
  'start_md_1': '2020-04-12',
  'end_md_1': '2020-05-30',
  'start_md_2': np.nan},
 {'country': 'KE',
  'start_md_1': '2020-03-24',
  'end_md_1': '2020-10-04',
  'start_md_2': np.nan},
 {'country': 'MX',
  'start_md_1': '2020-03-25',
  'end_md_1': '2020-10-06',
  'start_md_2': np.nan},
 {'country': 'NG',
  'start_md_1': '2020-03-27',
  'end_md_1': '2020-08-09',
  'start_md_2': np.nan},
 {'country': 'US',
  'start_md_1': '2020-03-21',
  'end_md_1': '2020-06-11',
  'start_md_2': '2020-11-26'}]

df_events = pd.DataFrame(event_dicts)

df_events['start_md_1'] = pd.to_datetime(df_events['start_md_1'])
df_events['end_md_1'] = pd.to_datetime(df_events['end_md_1'])
df_events['start_md_2'] = pd.to_datetime(df_events['start_md_2'])

df_agg = pd.read_pickle(DATA_DIR+'df_agg_cats.pickle')

In [48]:
def make_stars(val):
    if val<0.0001:
        return '****'
    elif val<0.001:
        return '***'
    elif val<0.01:
        return '**'
    elif val<0.05:
        return '*'
    else:
        return ''

def make_star_ste(value,ste):
    if value>0 and value-2*ste>0:
        return '*'
    elif value<0 and value+2*ste<0:
        return '*'
    else:
        return ''

In [49]:
weeks_2019 = list(df_agg.iloc[0]['volume_weekly_total'].index)[:52]
weeks_2020 = list(df_agg.iloc[0]['volume_weekly_total'].index)[52:]

In [50]:
l = []
for cnt, row in df_agg.iterrows():
    start_md = df_events.loc[df_events['country'] == row['country']].iloc[0]['start_md_1']
    end_md = df_events.loc[df_events['country'] == row['country']].iloc[0]['end_md_1']
    start_md2 = df_events.loc[df_events['country'] == row['country']].iloc[0]['start_md_2']
    
    for week in zip(row['volume_weekly_total'].index,row['volume_weekly_total'].values,row['volume_percent_weekly_total'].values):
        
        entry = {}

        entry['country'] = row['country']
        entry['category'] = row['category']
        

        if week[0] in weeks_2020:
            date = pd.to_datetime(week[0])

            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['volume_percent'] = week[2]
            entry['year'] = '2020'
            l.append(entry)

        elif week[0] in weeks_2019:
            date = pd.to_datetime(weeks_2020[weeks_2019.index(week[0])])
            
            if type(start_md2)!=pd._libs.tslibs.nattype.NaTType and date > start_md2:
                continue

            entry['k'] = math.floor(((date - start_md).days +7) / 7)
            entry['volume_total'] = week[1]
            entry['volume_percent'] = week[2]
            entry['year'] = '2019'
            l.append(entry)

df = pd.DataFrame(l)   

In [51]:
df = df.loc[(df['k'] >= -30) & (df['k'] <= 30)]
df = df.loc[(df['country'].isin(list(full_names.keys())))]
df['intervention_flag'] = df['k'].apply(lambda x: 1 if x >= 0 else 0)

In [52]:
cats = list(df['category'].unique())
k = 30

In [53]:
df_temp = df.loc[(df['k'] >= -k) & (df['k'] <= k)].copy()
df_temp['volume_total'] = df_temp['volume_total'].apply(np.log)

In [54]:
entries_list = []

for name, group in df_temp.groupby(['category']):
    print(name)
    
    entry = {}
    mod = smf.ols('volume_total ~ intervention_flag*year  + C(country)', data = group)
    res = mod.fit(cov_type='hc0')
    entry['model_degree'] = 0
    entry['category'] = name
    entry['alpha'] = res.params['intervention_flag:year[T.2020]']
    entry['ste'] = res.bse['intervention_flag:year[T.2020]']
    entry['pval'] = res.pvalues['intervention_flag:year[T.2020]']
    entry['r2'] = res.rsquared
    entries_list.append(entry)
    
    entry = {}
    mod = smf.ols('volume_total ~ intervention_flag*k*year  + C(country)', data = group)
    res = mod.fit(cov_type='hc0')
    entry['model_degree'] = 1
    entry['category'] = name
    entry['alpha'] = res.params['intervention_flag:year[T.2020]']
    entry['ste'] = res.bse['intervention_flag:year[T.2020]']
    entry['pval'] = res.pvalues['intervention_flag:year[T.2020]']
    entry['r2'] = res.rsquared
    entries_list.append(entry)
    
    entry = {}
    mod = smf.ols('volume_total ~ intervention_flag*k*year + intervention_flag*np.power(k,2)*year + C(country)', data = group)
    res = mod.fit(cov_type='hc0')
    entry['model_degree'] = 2
    entry['category'] = name
    entry['alpha'] = res.params['intervention_flag:year[T.2020]']
    entry['ste'] = res.bse['intervention_flag:year[T.2020]']
    entry['pval'] = res.pvalues['intervention_flag:year[T.2020]']
    entry['r2'] = res.rsquared
    entries_list.append(entry)
    
    

beef dish
bread and flatbread
cheese
chicken dish
cocktail
dessert
egg dish
fish dish
fruit
herb
lamb dish
pasta, pizza and noodle dish
pastry and bakery product
pie
pork dish
potato dish
rice dish
salad
sandwich
sauce
sausage
snack
soft drink
soup
spice
stew
vegetable and legume
wine, beer and liquor


In [55]:
df_res = pd.DataFrame(entries_list)

In [66]:
for j in range(28):
    print(str(j+1)+' &')
    for i in range(3):
        if i ==2:
            print(df_res.loc[df_res['model_degree']==i].sort_values(by = 'alpha', ascending = False)['category'].values[j] )
        else:
            print(df_res.loc[df_res['model_degree']==i].sort_values(by = 'alpha', ascending = False)['category'].values[j] + ' &')
    
    print('\\\\')

1 &
pastry and bakery product &
pastry and bakery product &
pastry and bakery product
\\
2 &
bread and flatbread &
bread and flatbread &
bread and flatbread
\\
3 &
pie &
pie &
pie
\\
4 &
dessert &
dessert &
dessert
\\
5 &
sauce &
potato dish &
potato dish
\\
6 &
chicken dish &
sauce &
sauce
\\
7 &
stew &
chicken dish &
cheese
\\
8 &
fruit &
stew &
stew
\\
9 &
vegetable and legume &
vegetable and legume &
egg dish
\\
10 &
egg dish &
pasta, pizza and noodle dish &
chicken dish
\\
11 &
rice dish &
fruit &
vegetable and legume
\\
12 &
potato dish &
cheese &
pasta, pizza and noodle dish
\\
13 &
cheese &
egg dish &
fruit
\\
14 &
pasta, pizza and noodle dish &
pork dish &
sausage
\\
15 &
spice &
rice dish &
pork dish
\\
16 &
herb &
beef dish &
beef dish
\\
17 &
pork dish &
sausage &
rice dish
\\
18 &
beef dish &
spice &
soup
\\
19 &
fish dish &
fish dish &
spice
\\
20 &
sausage &
herb &
salad
\\
21 &
salad &
snack &
fish dish
\\
22 &
soup &
soup &
snack
\\
23 &
snack &
salad &
cocktail
\\
24 

In [58]:
spearmanr(df_res.loc[df_res['model_degree']==0]['alpha'].values , 
          df_res.loc[df_res['model_degree']==2]['alpha'].values)

SpearmanrResult(correlation=0.9053092501368363, pvalue=3.644822598547773e-11)

In [59]:
spearmanr(df_res.loc[df_res['model_degree']==1]['alpha'].values , 
          df_res.loc[df_res['model_degree']==2]['alpha'].values)

SpearmanrResult(correlation=0.9649698960043787, pvalue=1.2473557427692476e-16)

In [60]:
spearmanr(df_res.loc[df_res['model_degree']==0]['alpha'].values , 
          df_res.loc[df_res['model_degree']==1]['alpha'].values)

SpearmanrResult(correlation=0.9512862616310892, pvalue=8.394478327545039e-15)